## Imports and definitions

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
from sys import path as syspath
syspath.append("functions/")

%load_ext autoreload
%autoreload 1
%aimport data_functions, model_functions
import os
from collections import OrderedDict
import pickle
# from data_functions import *
from model_functions import *
from copy import deepcopy
from Bio import SeqIO
bases = "acgt"
lett_to_index = dict(zip(bases,range(4)))
from scipy.special import logsumexp

genome = SeqIO.read("Ecoli/NC_000913.gbk", "genbank")

In [2]:
# assign position identities (inside == intragenic, outside == intergenic)
fttypes = set([ft.type for ft in genome.features])
genomeFlags = OrderedDict([])
take = OrderedDict([])
for fttype in fttypes:
    # print (fttype)
    take[fttype] = [ft for ft in genome.features if ft.type==fttype]
    nparts = [len(ft.location.parts) for ft in take[fttype]]
    # assert all parts within features are on the same strand
    for ft in take[fttype]:
        pts = ft.location.parts
        assert np.all([pt.strand==pts[0].strand for pt in pts])
    locs = [pt for ft in take[fttype] for pt in ft.location.parts]
    x = np.zeros((2,len(genome)))
    for il,lc in enumerate(locs):
        k = int(lc.strand<0)
        assert lc.start == lc.nofuzzy_start
        assert lc.end   == lc.nofuzzy_end
        x[k,lc.start:lc.end] = 1
    genomeFlags[fttype] = x


genFilter = OrderedDict([
    ("within", np.vstack([genomeFlags[k] for k in [
                        "misc_feature",
                        # "mobile_element",
                        # "repeat_region",
                        # "rep_origin",
                        # "tRNA",
                        # "STS",
                        # "tmRNA",
                        # "source"
                        # "rRNA",
                        # "ncRNA",
                        "CDS",
                        "gene",
                    ]]).sum(axis=0)>0),
    ("outside", np.vstack([genomeFlags[k] for k in [
                        "misc_feature",
                        "mobile_element",
                        "repeat_region",
                        # "rep_origin",
                        "tRNA",
                        "STS",
                        "tmRNA",
                        # "source"
                        "rRNA",
                        "CDS",
                        "gene",
                        "ncRNA",
                    ]]).sum(axis=0)==0),
    ])


## import model and transform sequence into numbers

In [3]:
# import model
with open("models/fitted_on_Pr.Pl.36N/model_[5]_extended", "rb") as f:
    theModel = pickle.load(f)

In [4]:
# treat sequences as if they come from 36N library
# (essentially, only affects which chemical potential is used)
treatAs = "36N"

In [5]:
# we are not interested to model how rc binding interferes with frw.
# but are using the rc strand as an independent strand.
# so, we can turn off the rc flag in the model
theModel["includeRC"] = False

In [6]:
# transform base pairs into indices
# offset is to get the same dimensionality of freeEs like the genome:
genOffset = sum(theModel["Layout"]) + theModel["spFlex"] 
numGen = np.array([lett_to_index[l] for l in genome.seq.lower()],dtype=np.int8)
numSeqs = np.array([
    np.hstack([numGen[-genOffset:],numGen      ]),
  3-np.hstack([numGen      ,numGen[: genOffset]])[::-1],
    
], dtype=np.int8)

assert np.all(numSeqs>=0)

## without pairwise interactions

In [7]:
# # without pairwise
# bricks = getBrickDict({treatAs: numSeqs}, theModel)[treatAs]
# freeEs = -logsumexp(-bricks, axis=-1)
# np.save("Ecoli/freeE_[5]_without_pwi",freeEs)
# del freeEs, bricks

In [8]:
freeEs = np.load("Ecoli/freeE_[5]_without_pwi.npy")

In [9]:
minimalValue = freeEs.min() # different for with and without pwi!

In [10]:
db = .2
bbs = np.arange(minimalValue-5, minimalValue+30, db)
bbs -= db/2

binCenters = (bbs[:-1]+bbs[1:])/2

In [11]:
def permuteMatrices(ms_, rndSeed = 1):
    leftm, rightm = ms_
    n1 = len(leftm)
    joined = np.vstack([leftm, rightm])
    np.random.seed(rndSeed)
    permIndex = np.random.permutation(np.arange(joined.shape[0]))
    permM = joined[permIndex]
    return permM[:n1], permM[n1:]

In [12]:
NrndShuffles = 100
rndCumulHist = {fl: np.nan*np.ones((NrndShuffles, len(binCenters))) for fl in genFilter}

In [13]:
for iRnd in range(100):
    tmpModel = deepcopy(theModel)
    tmpModel["matrices"] = permuteMatrices(theModel["matrices"], iRnd)
    bricks = getBrickDict({treatAs: numSeqs}, tmpModel,
#                           dinucl=True,
#                           dinuCoordsAndValues=(dinuDF["interaction"].values, dinuDF["value"].values)
                         )[treatAs]
    freeEs = -logsumexp(-bricks, axis=-1)
    rndFreeEnergy = { fl: freeEs[:,genFilter[fl]].flatten() for fl in genFilter }
    for fl in genFilter:
        rndCumulHist[fl][iRnd] = [(rndFreeEnergy[fl]<l).sum() for l in bbs[1:]]
    if os.path.isfile("stop.log"): break

In [14]:
with open("nbdumps/shuffled_matrix_genome_free_energies_without_pw.dmp","wb") as f:
    pickle.dump(rndCumulHist,f)

In [30]:
# with open("nbdumps/shuffled_matrix_genome_free_energies_without_pw.dmp","rb") as f:
#     saved = pickle.load(f)

In [31]:
# assert np.abs(saved[fl][iRnd]- rndCumulHist[fl][iRnd]).max()<1e-5

## with pairwise interactions

In [7]:
dinuDF = pd.read_csv("pairwise_interactions.csv", index_col=0)
dinuDF["interaction"] = list(map(eval, dinuDF["interaction"]))

In [8]:
# # with pairwise
# bricks = getBrickDict({treatAs: numSeqs}, theModel,
#                       dinucl=True,
#                       dinuCoordsAndValues=(dinuDF["interaction"].values, dinuDF["value"].values)
#                      )[treatAs]
# freeEs = -logsumexp(-bricks, axis=-1)
# # np.save("Ecoli/freeE_[5]_with_pwi",freeEs)
# # to free up memory
# # del freeEs, bricks

In [9]:
freeEs = np.load("Ecoli/freeE_[5]_with_pwi.npy")

In [10]:
minimalValue = freeEs.min() # different for with and without pwi!

In [11]:
db = .2
bbs = np.arange(minimalValue-5, minimalValue+30, db)
bbs -= db/2

binCenters = (bbs[:-1]+bbs[1:])/2

### (reindexed)

In [12]:
def permuteMatricesAndInteractions(ms_, interactions_, 
                                   rndSeed=1, 
                                   defaultSpacer=8,
                                   test=False):
    leftm, rightm = ms_
    n1 = len(leftm)
    n2 = len(rightm)
    joined = np.vstack([leftm, rightm])
    np.random.seed(rndSeed)
    if test:
        permIndex = np.arange(joined.shape[0])
    else:
        permIndex = np.random.permutation(np.arange(joined.shape[0]))
    permM = joined[permIndex]
    
    permIndex[permIndex>=n1]  = permIndex[permIndex>=n1]+defaultSpacer
    if test: print (permIndex)
    
#     interactionPos = np.array(interactions_)[:,[0,2]]
    
    reindexDict = dict(zip(
        list(range(n1))+list(range(n1+defaultSpacer, defaultSpacer+len(joined))),
        permIndex
                          ))
    permInteractions = np.vstack(interactions_)
    permInteractions[:,0] = [reindexDict[j] for j in permInteractions[:,0]]
    permInteractions[:,2] = [reindexDict[j] for j in permInteractions[:,2]]
    return permM[:n1], permM[n1:], permInteractions

In [14]:
NrndShuffles = 100

In [15]:
rndCumulHist = {fl: np.nan*np.ones((NrndShuffles, len(binCenters))) for fl in genFilter}

In [16]:
interactions = np.vstack(dinuDF["interaction"].values)

In [17]:
for iRnd in range(NrndShuffles):
    tmpModel = deepcopy(theModel)
    m1, m2, pi = permuteMatricesAndInteractions(theModel["matrices"], interactions, rndSeed=iRnd)
    tmpModel["matrices"] = [m1,m2]
    bricks = getBrickDict({treatAs: numSeqs}, tmpModel,
                          dinucl=True,
                          dinuCoordsAndValues=(pi, dinuDF["value"].values)
                         )[treatAs]
    freeEs = -logsumexp(-bricks, axis=-1)
    rndFreeEnergy = { fl: freeEs[:,genFilter[fl]].flatten() for fl in genFilter }
    for fl in genFilter:
        rndCumulHist[fl][iRnd] = [(rndFreeEnergy[fl]<l).sum() for l in bbs[1:]]
    if os.path.isfile("stop.log"): break
    break

In [18]:
# with open("nbdumps/shuffled_matrix_genome_free_energies_with_pwi_reindexed.dmp","wb") as f:
#     pickle.dump(rndCumulHist,f)

In [19]:
with open("nbdumps/shuffled_matrix_genome_free_energies_with_pwi_reindexed.dmp","rb") as f:
    saved = pickle.load(f)

In [20]:
assert np.abs(saved[fl][iRnd]- rndCumulHist[fl][iRnd]).max()<1e-5

###  not reindexed

In [21]:
def permuteMatricesAndInteractions(ms_, interactions_, 
                                   rndSeed=1, 
                                   defaultSpacer=8,
                                   test=False):
    leftm, rightm = ms_
    n1 = len(leftm)
    n2 = len(rightm)
    joined = np.vstack([leftm, rightm])
    np.random.seed(rndSeed)
    if test:
        permIndex = np.arange(joined.shape[0])
    else:
        permIndex = np.random.permutation(np.arange(joined.shape[0]))
    permM = joined[permIndex]
    
#     permIndex[permIndex>=n1]  = permIndex[permIndex>=n1]+defaultSpacer
#     if test: print (permIndex)
    
# #     interactionPos = np.array(interactions_)[:,[0,2]]
    
#     reindexDict = dict(zip(
#         list(range(n1))+list(range(n1+defaultSpacer, defaultSpacer+len(joined))),
#         permIndex
#                           ))
#     permInteractions = np.vstack(interactions_)
#     permInteractions[:,0] = [reindexDict[j] for j in permInteractions[:,0]]
#     permInteractions[:,2] = [reindexDict[j] for j in permInteractions[:,2]]
    return permM[:n1], permM[n1:], interactions_

In [22]:
NrndShuffles = 100

In [23]:
rndCumulHist = {fl: np.nan*np.ones((NrndShuffles, len(binCenters))) for fl in genFilter}

In [24]:
interactions = np.vstack(dinuDF["interaction"].values)

In [25]:
for iRnd in range(NrndShuffles):
    tmpModel = deepcopy(theModel)
    m1, m2, pi = permuteMatricesAndInteractions(theModel["matrices"], interactions, rndSeed=iRnd)
    tmpModel["matrices"] = [m1,m2]
    bricks = getBrickDict({treatAs: numSeqs}, tmpModel,
                          dinucl=True,
                          dinuCoordsAndValues=(pi, dinuDF["value"].values)
                         )[treatAs]
    freeEs = -logsumexp(-bricks, axis=-1)
    rndFreeEnergy = { fl: freeEs[:,genFilter[fl]].flatten() for fl in genFilter }
    for fl in genFilter:
        rndCumulHist[fl][iRnd] = [(rndFreeEnergy[fl]<l).sum() for l in bbs[1:]]
    if os.path.isfile("stop.log"): break
    break

In [26]:
# with open("nbdumps/shuffled_matrix_genome_free_energies_with_pwi.dmp","wb") as f:
#     pickle.dump(rndCumulHist,f)

In [27]:
with open("nbdumps/shuffled_matrix_genome_free_energies_with_pwi.dmp","rb") as f:
    saved = pickle.load(f)

In [28]:
assert np.abs(saved[fl][iRnd]- rndCumulHist[fl][iRnd]).max()<1e-5